# This notebook preprocesses the raw data
### from quality control to creating a clean aggregate data set

Input: raw data in the raw_data folder

Output: clean data in csv form in processed_data folder

Imports

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

Set global variables and propertes

In [2]:
raw_data_fname_str = ""
raw_data_folder = "raw_data"

processed_data_fname = "processed_data.csv"
processed_data_folder = "processed_data"
processed_data_file = os.path.join(processed_data_folder, processed_data_fname)

Create test data frame

In [3]:
processed_data = pd.DataFrame({"subject": [], "trial": [], "valid": [], 
                               "choices first stage": [], "choices second stage": [], 
                               "RTs first stage": [], "RTs second stage": [], 
                               "states first stage": [], "states second stage": [], 
                               "outcomes first stage": [], "outcomes second stage": []})

Save as CSV

In [4]:
processed_data.to_csv(processed_data_file)

Read the CSV to test that writing worked

In [5]:
test_data = pd.read_csv(processed_data_file)

print(test_data)

Empty DataFrame
Columns: [Unnamed: 0, subject, trial, valid, choices first stage, choices second stage, RTs first stage, RTs second stage, states first stage, states second stage, outcomes first stage, outcomes second stage]
Index: []
